Plot de regiones interactivo, agregar segmentos a reg y usar como parametros para OC

In [1]:
import ipywidgets as widgets
from ipywidgets.widgets import HBox,VBox
import matplotlib.pyplot as plt
import matplotlib as mpl
from ipywidgets import interactive, interactive_output
import numpy as np
from scipy.optimize import curve_fit
from IPython.display import clear_output
import numpy as np
from scipy.interpolate import interp1d
from astropy.constants import c
import astropy.io.fits as pf


cvel = c.to('km/s').value

In [2]:
c=widgets.FloatRangeSlider(
    value=[5, 7.5],
    min=0,
    max=10.0,
    step=0.1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)

add_reg_button = widgets.Button(value=False,
    description='add',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='add region',
    icon='sliders',
    width='200px')
reg = []
def add_to_regions(b, reg = reg):
    reg.append(c.value)
    return reg

add_reg_button.on_click(add_to_regions)

display(c,add_reg_button)


FloatRangeSlider(value=(5.0, 7.5), continuous_update=False, description='Test:', max=10.0, readout_format='.3f…

Button(description='add', icon='sliders', style=ButtonStyle(), tooltip='add region')

In [ ]:
reg

In [ ]:
#plot input models and spec

In [3]:
from spectools import utils

In [4]:
def ascii_spec(filename):
    a = np.genfromtxt(filename,delimiter = '  ')
    w = a[:,0]
    f = a[:,-1]
    return w,f

def open_shift(A : str, B : str , va : (float, np.float64), vb : (float, np.float64), 
               a : (float,np.float64) = None, b : (float,np.float64) = None ):
    wa,fa = ascii_spec(A)
    wb,fb = ascii_spec(B)
    #Aplico cotas 
    if a is not None:
        fa = fa[wa>a]
        wa = wa[wa>a]
        fb = fb[wb>a]
        wb = wb[wb>a]
    if b is not None:
        fa = fa[wa<b]
        wa = wa[wa<b]
        fb = fb[wb<b]
        wb = wb[wb<b]
    fa_shift = shift(wa, fa, va)
    fb_shift = shift(wb, fb, vb, wa)#va a resamplear con el eje de wa, 
                                    #si son modelos del mismo programa 
                                    #deberían ser similares, pero el 
                                    #doppler shift puede cambiarlo
    #Me aseguro que los arrays tengan el mismo largo
    if len(fb_shift) > len(fa_shift):
        arr_len = len(fa_shift)
    else: arr_len = len(fb_shift)
    
    w1=wa[:arr_len]
    fa_s=fa_shift[:arr_len]
    fb_s=fb_shift[:arr_len]
    # A partir de acá los dos modelos tienen el mismo eje de dispersion
    return w1,fa_s,fb_s

def shift(wvl,flux,v,new_wvl = None):
    wlprime = wvl * (1.0 + v / cvel)
    if new_wvl is not None:
        wvl = new_wvl
    nflux = interp1d(wlprime, flux, bounds_error=False, fill_value=1)(wvl)
    return nflux

def open_file(filename): #for 1Dspecs ?
    hdu = pf.open(filename)
    data = hdu[0].data
    header = hdu[0].header
    wave = wavelength(header)
    return wave,data

def wavelength(header):
    wave = [header['CRVAL1'] + header['CDELT1'] * i for i in range(header['NAXIS1'])]
    return np.array(wave)

def mask_line(x,y,a,b):
    x_ma=np.ma.masked_outside(x,a,b)
    mask = np.ma.getmask(x_ma)
    f_ma=np.ma.array(y,mask=mask)    
    return x_ma,f_ma

In [ ]:
#A = 'T350g370.asc'
#B = 'T300g380.asc'
#v1=-100
#v2=300

#w1, fa_s, fb_s = open_shift(A,B,v1,v2,b=7000)

#dils = [0.6,0.65,0.7,0.75,0.8,0.85]
#lines=[(4080,4112),(4330,4346),(4845,4875),(4019,4035),(5868,5885),(4682,4693),(4197,4203),(4537,4543),(4682,4686),(5400,5417)]
#Esto son etiquetas para el mapa de color

#labels_y = [str(d) for d in dils]
#labels_x = [str(l) for l in lines]

tests = OC_dils(w1,fa_s,fb_s,wo,fo,dils,lines)

In [5]:
wo,fo=open_file('HD37043_20170408_201940_M_V85000.fits')
fo = fo[0]
A = 'T350g370.asc'
B = 'T300g380.asc'
v1=-100
v2=300

w1, fa_s, fb_s = open_shift(A,B,v1,v2,b=7000)

IMPORTANT The spectrum is NOT corrected from barycentric velocity                [astropy.io.fits.card]
IMPORTANT Use the keyword I_VBAR to correct from barycentric velocity            [astropy.io.fits.card]
IMPORTANT This fits file includes a normalized and the orig. merged spect.       [astropy.io.fits.card]
VERY IMPORTANT ** The I_SPC keyword below is only orientative, no definitive. ** [astropy.io.fits.card]
VERY IMPORTANT ** Please, use it carefully, not as a reference !!!            ** [astropy.io.fits.card]


In [13]:
fit_range1 = widgets.FloatRangeSlider(
    value=[4000, 5000],
    min=wo[1],
    max=wo[-2],
    step=0.1,
    description='Range to fit:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
    layout={'width':'600px'},
    style = {'description_width': 'initial'}
)

plot_range1 = widgets.FloatRangeSlider(
    value=[wo[1], wo[-2]],
    min=wo[0],
    max=wo[-1],
    step=0.1,
    description='Range to plot:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
    layout={'width':'600px'},
    style = {'description_width': 'initial'}
)

dilution0 = widgets.FloatSlider(
    value= 0.5,
    min = 0.01,
    max = 1.,
    step = 0.01,
    description='Dilution',
    disabled=False,
    readout=True,
    readout_format='.3f',
    orientation='vertical')

ylim = widgets.FloatRangeSlider(
    value= [0.5,1.2],
    min = 0.01,
    max = 1.5,
    step = 0.01,
    description='ylim',
    disabled=False,
    readout=True,
    readout_format='.3f',
    orientation='vertical')

In [20]:
def d(rang,ab,dil,ylim):
    fig,ax=plt.subplots()

#    global p0,v,w#,fig,ax
    ax.set_ylim(ylim[0], ylim[1])
    ax.set_xlim(rang[0],rang[1])
    v,w=mask_line(x,y,ab[0],ab[1])
#    p0=[c,inten,m,sigma]
    ax.plot(x, y,c='grey',linewidth=1,label='observed')
#    ax.plot(w1,f1*dil+1-dil,c='cyan',linewidth=0.7,alpha=0.5,label='model1')
#    ax.plot(w1,f2*(1-dil)+dil,c='magenta',linewidth=0.7,alpha=0.5,label='model2')
    ax.plot(v,w,c='green',linewidth=0.9,label='selected for fit')
#    plt.legend()

add_reg_button = widgets.Button(value=False,
    description='add',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='add region',
    icon='sliders',
    width='200px')

reg = []
x=wo
y=fo
f1=fa_s
f2=fb_s

def add_to_regions(b, reg = reg):
    reg.append(fit_range1.value)
    return reg

add_reg_button.on_click(add_to_regions)
wids1 = widgets.VBox([plot_range1,fit_range1,
                     HBox([add_reg_button])
                    ],)
interactive_plot1 = interactive_output(d,{'rang':plot_range1,'ab':fit_range1,'dil':dilution0,'ylim':ylim})
app = widgets.AppLayout(header=None,
          left_sidebar=wids1,
          center=widgets.VBox([dilution0,ylim]),
          right_sidebar=interactive_plot1,
          footer=None,
          pane_widths=['620px','90px','600px'],
          )
app


AppLayout(children=(VBox(children=(FloatRangeSlider(value=(3763.75, 9006.34375), continuous_update=False, desc…

In [18]:
reg

[(4000.05, 5000.05), (5423.45, 6423.45), (5423.45, 7367.85)]

In [ ]:
rang = (wo[0],wo[-1])
ab= [3800,4000]
x=wo
y=fo
f1=fa_s
f2=fb_s
fig,ax=plt.subplots()

ax.set_ylim(0.5, 1.03)
ax.set_xlim(rang[0],rang[1])
v,w=mask_line(x,y,ab[0],ab[1])
#    p0=[c,inten,m,sigma]
ax.plot(x, y,c='grey',linewidth=1,label='observed')
ax.plot(w1,f1,c='cyan',linewidth=0.7,alpha=0.5,label='model1')
ax.plot(w1,f2,c='magenta',linewidth=0.7,alpha=0.5,label='model2')
ax.plot(v,w,c='green',linewidth=0.9,label='selected for fit')